### Content

* I. Initial

* II. Load Corpus

* III. Train GAT

* IV. Train Conv

* V. Evaluation

In [ ]:
from google.colab import drive
drive.flush_and_unmount()

Drive not mounted, so nothing to flush and unmount.


In [ ]:
rm -rf /content/*

# I. Initial

In [ ]:
%%bash
rm -rf GCAT
git clone https://github.com/hmthanh/GCAT.git

Cloning into 'GCAT'...
Checking out files: 100% (120/120), done.


In [ ]:
!yes | cp --i /content/GCAT/* /content/

cp: -r not specified; omitting directory '/content/GCAT/data'
cp: -r not specified; omitting directory '/content/GCAT/output'


In [ ]:
%%bash
rm -rf /content/data/
rm -rf /content/output/
rm -r /content/sample_data/

mv /content/GCAT/* /content/
rm -r /content/GCAT/

rm: cannot remove '/content/sample_data/': No such file or directory


Load Google Drive if you want to save model on Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


## 1. Config

In [ ]:
#@title Cấu hình cài đặt quá trình huấn luyện { run: "auto" }

#@markdown Siêu tham số

dataset = "FB15k" #@param ["WN18RR", "WN18", "FB15k", "FB15k-237"] {allow-input: false}
cuda = True #@param {type: "boolean"}
save_gdrive = True #@param {type: "boolean"}
data_folder = "./data" #@param {type: "string"}
output_folder="./output" #@param {type: "string"}
drive_folder="/content/drive/My Drive" #@param {type: "string"}

epochs_gat = 3000 #@param {type: "number"}
epochs_conv = 150 #@param {type: "number"}
weight_decay_gat = float(1e-5) #@param {type:"raw"}
weight_decay_conv = float(1e-5) #@param {type:"raw"}
pretrained_emb = False #@param {type: "boolean"}
embedding_size = 50 #@param {type: "slider", min: 20, max: 200}
lr = float(1e-3) #@param {type:"raw"}
get_2hop = True #@param {type: "boolean"}
use_2hop = True #@param {type: "boolean"}
partial_2hop = True #@param {type: "boolean"}

#@markdown --- Tham số cho mô hình GAT
batch_size_gat = 272115 #@param {type: "number"}
valid_invalid_ratio_gat = 2 #@param {type: "slider", min: 1, max: 10}
drop_GAT = 0.3  #@param {type:"raw"}
alpha = 0.2  #@param {type:"raw"}
entity_out_dim = [100, 200]  #@param {type:"raw"}
nheads_GAT = [2, 2]  #@param {type:"raw"}
margin = 1 #@param {type: "slider", min: 1, max: 10}

#@markdown --- Tham số cho mô hình ConvKB
batch_size_conv = 128  #@param {type: "slider", min: 16, max: 512, step:16}
alpha_conv = 0.2  #@param {type:"raw"}
valid_invalid_ratio_conv = 40
out_channels = 50 #@param {type: "slider", min: 50, max: 1000, step:50}
drop_conv = 0.3  #@param {type:"raw"}

from create_config import Config

config = Config(dataset=dataset, data_folder=data_folder,cuda=cuda,output_folder=output_folder,save_gdrive=save_gdrive,drive_folder=drive_folder,epochs_gat=epochs_gat,epochs_conv=epochs_conv,weight_decay_gat=weight_decay_gat,weight_decay_conv=weight_decay_conv,pretrained_emb=pretrained_emb,embedding_size=embedding_size,lr=lr,get_2hop=get_2hop,use_2hop=use_2hop,partial_2hop=partial_2hop,batch_size_gat=batch_size_gat,valid_invalid_ratio_gat=valid_invalid_ratio_gat,drop_GAT=drop_GAT,alpha=alpha,entity_out_dim=entity_out_dim,nheads_GAT=nheads_GAT,margin=margin,batch_size_conv=batch_size_conv,alpha_conv=alpha_conv,valid_invalid_ratio_conv=valid_invalid_ratio_conv,out_channels=out_channels,drop_conv=drop_conv)

config.dumps_config()

## 2. WN18RR

In [ ]:
#@title Cấu hình cài đặt WN18RR { run: "auto" }

#@markdown Siêu tham số

dataset = "WN18" #@param ["WN18RR", "WN18", "FB15k", "FB15k-237", "Thanh"] {allow-input: false}
cuda = False #@param {type: "boolean"}
data_folder = "./data"
output_folder="./output" #@param {type: "string"}
drive_folder="/content/drive/My Drive" #@param {type: "string"}
save_gdrive = True #@param {type: "boolean"}
epochs_gat = 3600 #@param {type: "number"}
epochs_conv = 200 #@param {type: "number"}
weight_decay_gat = float(5e-6) #@param {type:"raw"}
weight_decay_conv = float(1e-5) #@param {type:"raw"}
pretrained_emb = False #@param {type: "boolean"}
embedding_size = 50 #@param {type: "slider", min: 20, max: 200}
lr = float(1e-3) #@param {type:"raw"}
get_2hop = True #@param {type: "boolean"}
use_2hop = True #@param {type: "boolean"}
partial_2hop = False #@param {type: "boolean"}

#@markdown --- Tham số cho mô hình GAT
batch_size_gat = 86835 #@param {type: "number"}
valid_invalid_ratio_gat = 2 #@param {type: "slider", min: 1, max: 10}
drop_GAT = 0.3  #@param {type:"raw"}
alpha = 0.2  #@param {type:"raw"}
entity_out_dim = [100, 200]  #@param {type:"raw"}
nheads_GAT = [2, 2]  #@param {type:"raw"}
margin = 5 #@param {type: "slider", min: 1, max: 10}

#@markdown --- Tham số cho mô hình ConvKB
batch_size_conv = 128  #@param {type: "slider", min: 16, max: 512, step:16}
alpha_conv = 0.2  #@param {type:"raw"}
valid_invalid_ratio_conv = 40
out_channels = 500 #@param {type: "slider", min: 100, max: 1000, step:100}
drop_conv = 0.0  #@param {type:"raw"}

from create_config import Config

config = Config(dataset=dataset,data_folder=data_folder,cuda=cuda,output_folder=output_folder,save_gdrive=save_gdrive,drive_folder=drive_folder,epochs_gat=epochs_gat,epochs_conv=epochs_conv,weight_decay_gat=weight_decay_gat,weight_decay_conv=weight_decay_conv,pretrained_emb=pretrained_emb,embedding_size=embedding_size,lr=lr,get_2hop=get_2hop,use_2hop=use_2hop,partial_2hop=partial_2hop,batch_size_gat=batch_size_gat,valid_invalid_ratio_gat=valid_invalid_ratio_gat,drop_GAT=drop_GAT,alpha=alpha,entity_out_dim=entity_out_dim,nheads_GAT=nheads_GAT,margin=margin,batch_size_conv=batch_size_conv,alpha_conv=alpha_conv,valid_invalid_ratio_conv=valid_invalid_ratio_conv,out_channels=out_channels,drop_conv=drop_conv)

config.dumps_config()

config

dataset: WN18
data_folder: ./data
output_folder: ./output
drive_folder: /content/drive/My Drive
save_gdrive: True
cuda: False
epochs_gat: 3600
epochs_conv: 200
weight_decay_gat: 5e-06
weight_decay_conv: 1e-05
pretrained_emb: False
embedding_size: 50
lr: 0.001
get_2hop: True
use_2hop: True
partial_2hop: False
batch_size_gat: 86835
valid_invalid_ratio_gat: 2
drop_GAT: 0.3
alpha: 0.2
entity_out_dim: [100, 200]
nheads_GAT: [2, 2]
margin: 5
batch_size_conv: 128
alpha_conv: 0.2
valid_invalid_ratio_conv: 40
out_channels: 500
drop_conv: 0.0

# II. Create corpus


In [ ]:
%%bash
python 1_create_corpus.py

number of unique_entities -> 40943
number of unique_entities -> 7802
number of unique_entities -> 7845
test_triples (5000, 3)
test_adjacency_mat (3, 5000)
entity2id 40943
relation2id 18
headTailSelector 18
unique_entities_train (40943,) 

Initialised relations and entities randomly
Graph created
length of graph keys is  40504
time taken  9517.742918252945
length of neighbors dict is  40416
Total triples count 151442, training triples 141442, validation_triples 5000, test_triples 5000
1. Created Corpus Successfully !


In [ ]:
torch.cuda.max_memory_allocated(

## III. Training GAT encoder

In [ ]:
import torch
torch.backends.cudnn.deterministic=True
print(torch.cuda.memory_allocated())
print(torch.cuda.max_memory_allocated())

0
0


In [ ]:
!export CUDA_LAUNCH_BLOCKING=1

In [ ]:
!nvidia-smi

Mon Sep  7 11:48:18 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    28W / 250W |     10MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!python 2_training_encoder.py

Loading corpus
Defining model
Defining loss
length of unique_entities  40943
Number of epochs 3600

epoch->  0
tcmalloc: large alloc 1641709568 bytes == 0xb6516000 @  0x7fe533b46b6b 0x7fe533b66379 0x7fe4e82d392e 0x7fe4e82d5946 0x7fe5206b1bb3 0x7fe52016617a 0x7fe520167293 0x7fe520667edf 0x7fe5201f56a6 0x7fe5201f67b0 0x7fe52044d1c9 0x7fe51fdee689 0x7fe52052c919 0x7fe5201f809b 0x7fe5205e2a08 0x7fe51fdee689 0x7fe52052c709 0x7fe5221defc7 0x7fe51fdee689 0x7fe52052c709 0x7fe52fca3a0e 0x50a7f5 0x50cfd6 0x509015 0x594b01 0x59fe1e 0x50d596 0x507f24 0x509202 0x594b01 0x54ac61
tcmalloc: large alloc 1641709568 bytes == 0x159686000 @  0x7fe533b46b6b 0x7fe533b66379 0x7fe4e82d392e 0x7fe4e82d5946 0x7fe5206b1bb3 0x7fe52016617a 0x7fe520167293 0x7fe520667edf 0x7fe5201f56a6 0x7fe5201f67b0 0x7fe52044d1c9 0x7fe51fdee689 0x7fe52052c919 0x7fe5201f809b 0x7fe5205e2a08 0x7fe51fdee689 0x7fe52052c709 0x7fe5221defc7 0x7fe51fdee689 0x7fe52052c709 0x7fe52fca3a0e 0x50a7f5 0x50cfd6 0x509015 0x594b01 0x59fe1e 0x50d596 0x

# III Training conv decoder

In [ ]:
!python 3_training_decoder.py

Loading corpus
Defining model
Loading GAT encoder
Only Conv model trained
Number of epochs 1

epoch->  0
tcmalloc: large alloc 2115993600 bytes == 0x73a8000 @  0x7fe900925b6b 0x7fe900945379 0x7fe8b50b292e 0x7fe8b50b4946 0x7fe8ed1cf4b1 0x7fe8ed1bf519 0x7fe8ed1c9f2f 0x7fe8ed1ce06e 0x7fe8ed1c7126 0x7fe8ed1c7baf 0x7fe8ed3c9016 0x7fe8ecbccc53 0x7fe8ed32c01a 0x7fe8eef3518f 0x7fe8ecbccc53 0x7fe8ed32c01a 0x7fe8ecd39607 0x7fe8ed3c283c 0x7fe8eef5eca1 0x7fe8ecbccbb8 0x7fe8ed32cc00 0x7fe8ecd310a7 0x7fe8ed3c254d 0x7fe8eef5eeeb 0x7fe8ecbccd0f 0x7fe8ed32d1f1 0x7fe8ecd30ec6 0x7fe8ed3c2c06 0x7fe8eefe49a1 0x7fe8ecbccc53 0x7fe8ed32bada
tcmalloc: large alloc 2099200000 bytes == 0x855a0000 @  0x7fe900925b6b 0x7fe900945379 0x7fe8b50b292e 0x7fe8b50b4946 0x7fe8ecfa59e5 0x7fe8ed22aaf3 0x7fe8ed21bf97 0x7fe8ed21bc7d 0x7fe8ed21bf97 0x7fe8ed326a1a 0x7fe8ed1da694 0x7fe8ed1c7c7b 0x7fe8ed3c9016 0x7fe8ecbccc53 0x7fe8ed32c01a 0x7fe8eef3518f 0x7fe8ecbccc53 0x7fe8ed32c01a 0x7fe8ecd39607 0x7fe8ed3c283c 0x7fe8eef5eca1 0x7f

# V. Evaluation

### 1. Evaluating

In [ ]:
!python 4_evalution.py

Loading corpus
Loading model
Sampled indices Hello
test set length  49
0
4. Evaluation Successfully !


### 2. Load result